In [1]:
import numpy as np

In [2]:
suffix = ["fp", "bfp", "bfp-sorted"]
# suffix = ["fp", "bfp"]
iter_count = 5
layer_count = 1

def get_np(path):
    with open(path, "rb") as f:
        return np.load(f, allow_pickle=False)

def get_sparsity(array: np.ndarray, threshold) -> np.ndarray:
    results = []

    for row in array:
        total = len(row)
        over_thresholds = np.where(row >= threshold)
        if len(over_thresholds) > 1:
            raise ValueError(f"invalid tuple size: {len(over_thresholds)}")

        zeros = len(np.where(row >= threshold)[0])
        results.append(zeros / total)
        # print(f"{(zeros / total * 100):.5f}")

    return np.array(results)
        

"""
with open(f"{path}/i{iter}-l{layer_index}-fwd-lhs-ori-E.npy", "wb") as f:
                   np.save(f, lhs.ori_E.numpy(), allow_pickle=False)

            with open(f"{path}/i{iter}-l{layer_index}-fwd-lhs-bfp-E.npy", "wb") as f:
                   np.save(f, lhs.bfp_E.numpy(), allow_pickle=False)

            with open(f"{path}/i{iter}-l{layer_index}-fwd-rhs-ori-E.npy", "wb") as f:
                   np.save(f, rhs.ori_E.numpy(), allow_pickle=False)

            with open(f"{path}/i{iter}-l{layer_index}-fwd-rhs-bfp-E.npy", "wb") as f:
                   np.save(f, rhs.bfp_E.numpy(), allow_pickle=False)
"""

names = ["fwd", "grad-w", "grad-act"]

for i in range(iter_count):
    for l in range(layer_count):
    
        # name = "fwd"
        # name = "grad-w"
        # name = "grad-act"
        for name in names:
            # print(name)
            path = f"/home/yskim/projects/sparse-bfp/outputs/i{i}-l{l}"

            lhs_ori = get_np(f"{path}-{name}-lhs-ori-E.npy")
            lhs_bfp = get_np(f"{path}-{name}-lhs-bfp-E.npy")

            rhs_ori = get_np(f"{path}-{name}-rhs-ori-E.npy")
            rhs_bfp = get_np(f"{path}-{name}-rhs-bfp-E.npy")


            lhs_offset = lhs_bfp - lhs_ori
            rhs_offset = rhs_bfp - rhs_ori

            threshold = 8
            lhs_sparsity = get_sparsity(lhs_offset, threshold)
            rhs_sparsity = get_sparsity(rhs_offset, threshold)

            print(f"[{name} (iter: {i}, layer: {l})] lhs: {lhs_ori.shape}, rhs: {rhs_ori.shape}")
            print(f"*avg sparsity, lhs: {(np.mean(lhs_sparsity) * 100):.2f}%, rhs: {(np.mean(rhs_sparsity) * 100):.2f}%")
            print(f"*std sparsity, lhs: {(np.std(lhs_sparsity) * 100):.2f}%, rhs: {(np.std(rhs_sparsity) * 100):.2f}%")
            print(f"*max sparsity, lhs: {(np.max(lhs_sparsity) * 100):.2f}%, rhs: {(np.max(rhs_sparsity) * 100):.2f}%")
            print(f"*min sparsity, lhs: {(np.min(lhs_sparsity) * 100):.2f}%, rhs: {(np.min(rhs_sparsity) * 100):.2f}%")
            continue

            lhs_dict = {}
            rhs_dict = {}

            for offset in lhs_offset:
                if offset not in lhs_dict.keys():
                    lhs_dict[offset] = 1
                else:
                    lhs_dict[offset] += 1

            for offset in rhs_offset:
                if offset not in rhs_dict.keys():
                    rhs_dict[offset] = 1
                else:
                    rhs_dict[offset] += 1

            print("[LHS]")
            lhs_total = 0
            lhs_zero = 0    
            for key in lhs_dict.keys():
                value = lhs_dict[key]
                print(f"{key}: {value}")
                if key >= 4:
                    lhs_zero += value
                lhs_total += value
            print(f"total: {lhs_total}, zeros: {lhs_zero} ({(lhs_zero / lhs_total * 100.0):.2f}%)")
            

            print("[RHS]")
            rhs_total = 0
            rhs_zero = 0    
            for key in rhs_dict.keys():
                value = rhs_dict[key]
                print(f"{key}: {value}")
                if key >= 4:
                    rhs_zero += value
                rhs_total += value
            print(f"total: {rhs_total}, zeros: {rhs_zero} ({(rhs_zero / rhs_total * 100.0):.2f}%)")

            print()
    print()



[fwd (iter: 0, layer: 0)] lhs: (32, 1280), rhs: (1000, 1280)
*avg sparsity, lhs: 22.67%, rhs: 0.79%
*std sparsity, lhs: 2.11%, rhs: 0.26%
*max sparsity, lhs: 28.36%, rhs: 1.56%
*min sparsity, lhs: 18.44%, rhs: 0.00%
[grad-w (iter: 0, layer: 0)] lhs: (1000, 32), rhs: (1280, 32)
*avg sparsity, lhs: 30.01%, rhs: 22.70%
*std sparsity, lhs: 16.84%, rhs: 7.25%
*max sparsity, lhs: 84.38%, rhs: 56.25%
*min sparsity, lhs: 0.00%, rhs: 3.12%
[grad-act (iter: 0, layer: 0)] lhs: (32, 1000), rhs: (1280, 1000)
*avg sparsity, lhs: 26.09%, rhs: 0.79%
*std sparsity, lhs: 6.05%, rhs: 0.29%
*max sparsity, lhs: 40.70%, rhs: 2.10%
*min sparsity, lhs: 12.50%, rhs: 0.10%


FileNotFoundError: [Errno 2] No such file or directory: '/home/yskim/projects/sparse-bfp/outputs/i0-l1-fwd-lhs-ori-E.npy'

In [ ]:
# output_fp = get_np(f"/home/yskim/projects/sparse-bfp/outputs/fwd-2-fp.npy")
# output_bfp = get_np(f"/home/yskim/projects/sparse-bfp/outputs/fwd-2-bfp.npy")
# print(np.argmax(output_fp))
# print(np.argmax(output_bfp))